In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving Texol Kireka Data.xlsx to Texol Kireka Data (1).xlsx


In [ ]:
# Assuming the file name is 'Texol Kireka Data.xlsx' and is present in the uploaded dictionary
file_name = list(uploaded.keys())[0]

# Read data from the uploaded Excel file
df = pd.read_excel(file_name)

# Convert 'DATE' to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M')

# Set 'DATE' as the index
df.set_index('Date', inplace=True)

# Extract customer type from Card Number
df['Customer Type'] = df['Card Number'].str[:4]

# Calculate customer frequencies
customer_frequencies = df['Customer Name'].value_counts()

# Calculate total amount by customer type
pivot_customer_type = df.pivot_table(values='AMOUNT', index='Customer Type', aggfunc='sum')

# Create pivot tables
pivot_hour = df.pivot_table(values='AMOUNT', index=df.index.hour, aggfunc='sum')
pivot_customer = df.pivot_table(values='AMOUNT', index='Customer Name', aggfunc='sum')

# Identify customers with amounts over 100,000
high_value_customers = df[df['AMOUNT'] > 100000]
high_value_customers['Category'] = high_value_customers['Customer Type'].apply(lambda x: 'TEXI' if x == 'TEXI' else ('TEXB' if x == 'TEXB' else 'TEXC'))

# Time series forecasting
# Resample the data to get daily totals
daily_data = df.resample('D').sum()

<ipython-input-34-8b1f7f84a8c4>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Assuming the file name is 'Texol Kireka Data.xlsx' and is present in the uploaded dictionary
file_name = list(uploaded.keys())[0]

# Read data from the uploaded Excel file
df = pd.read_excel(file_name)

# Convert 'DATE' to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M')

# Set 'DATE' as the index
df.set_index('Date', inplace=True)

# Extract customer type from Card Number
df['Customer Type'] = df['Card Number'].str[:4]

# Calculate customer frequencies
customer_frequencies = df['Customer Name'].value_counts()

# Calculate total amount by customer type
pivot_customer_type = df.pivot_table(values='AMOUNT', index='Customer Type', aggfunc='sum')

# Create pivot tables
pivot_hour = df.pivot_table(values='AMOUNT', index=df.index.hour, aggfunc='sum')
pivot_customer = df.pivot_table(values='AMOUNT', index='Customer Name', aggfunc='sum')

# Identify customers with amounts over 100,000
high_value_customers = df[df['AMOUNT'] > 100000]
high_value_customers['Category'] = high_value_customers['Customer Type'].apply(lambda x: 'TEXI' if x == 'TEXI' else ('TEXB' if x == 'TEXB' else 'TEXC'))

# Time series forecasting
# Resample the data to get daily totals
daily_data = df.resample('D').sum()

# Apply the Simple Exponential Smoothing model
model = SimpleExpSmoothing(daily_data['AMOUNT'])
fit = model.fit()

# Forecast for the next 30 days
forecast = fit.forecast(30)

# Combine actual and forecasted data for plotting
forecast_df = daily_data.copy()
#forecast_df = forecast_df.append(forecast.rename('Forecast'))

# Interactive Plot for Amount by Hour
fig_hour = px.line(pivot_hour, title='Total Amount by Hour', labels={'index': 'Hour of the Day', 'value': 'Total Amount'})

# Interactive Plot for Amount by Customer
fig_customer = px.bar(pivot_customer, title='Total Amount by Customer', labels={'index': 'Customer Name', 'value': 'Total Amount'})

# Interactive Plot for Customer Frequencies
fig_frequencies = px.bar(customer_frequencies, title='Customer Frequencies', labels={'index': 'Customer Name', 'value': 'Frequency'})

# Interactive Plot for Amount by Customer Type
fig_customer_type = px.bar(pivot_customer_type, title='Total Amount by Customer Type', labels={'index': 'Customer Type', 'value': 'Total Amount'})

# Interactive Plot for High Value Customers
fig_high_value = px.bar(high_value_customers, x='Customer Name', y='AMOUNT', color='Category', title='High Value Customers (Amount > 100,000)', labels={'AMOUNT': 'Amount', 'Customer Name': 'Customer Name'})

# Interactive Plot for Sales Forecasting
fig_forecast = go.Figure()
fig_forecast.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['AMOUNT'], mode='lines', name='Actual'))
fig_forecast.add_trace(go.Scatter(x=forecast.index, y=forecast, mode='lines', name='Forecast', line=dict(dash='dash')))
fig_forecast.update_layout(title='Sales Forecast', xaxis_title='Date', yaxis_title='Amount')

# Display the plots
fig_hour.show()
fig_customer.show()
fig_frequencies.show()
fig_customer_type.show()
fig_high_value.show()
fig_forecast.show()

# Save the plots as HTML files
pio.write_html(fig_hour, file='fig_hour.html', auto_open=False)
pio.write_html(fig_customer, file='fig_customer.html', auto_open=False)
pio.write_html(fig_frequencies, file='fig_frequencies.html', auto_open=False)
pio.write_html(fig_customer_type, file='fig_customer_type.html', auto_open=False)
pio.write_html(fig_high_value, file='fig_high_value.html', auto_open=False)
pio.write_html(fig_forecast, file='fig_forecast.html', auto_open=False)

# Display data frames for analysis
print("Customer Frequencies:")
print(customer_frequencies)

print("\nTotal Amount by Customer Type:")
print(pivot_customer_type)

print("\nHigh Value Customers (Amount > 100,000):")
print(high_value_customers)


<ipython-input-39-91e4bffc5ebe>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



Customer Frequencies:
Customer Name
Vicent  Mayende                55
Ssendawule Hissein             42
Amanya Derrick                 42
John Chris Muganga             42
Ssentamu Ismael  Kyagulanyi    41
                               ..
Dononzio  Munage                1
Darius  Bashimiki               1
Musa Seguya                     1
Sharfik  Bukenya                1
MALINGA  BRIAN                  1
Name: count, Length: 1048, dtype: int64

Total Amount by Customer Type:
                 AMOUNT
Customer Type          
TEXB           28525756
TEXC            3700000
TEXI           80884955

High Value Customers (Amount > 100,000):
                             Customer Name     Card Number   AMOUNT  \
Date                                                                  
2024-06-01 23:22:30       Haruna Ssempijja  TEXI0000001808   230000   
2024-06-01 16:35:52        Sirajje Muteesa  TEXI0000007087   200000   
2024-06-01 12:34:05        Henry  Sempebwa  TEXI0000007072   300000   
2

In [ ]:
from google.colab import files

# Download the HTML files
files.download('fig_hour.html')
files.download('fig_customer.html')
files.download('fig_frequencies.html')
files.download('fig_customer_type.html')
files.download('fig_high_value.html')
files.download('fig_forecast.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>